In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reading dataset file
dataset = pd.read_csv('combined_data_1.txt',header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
dataset.head(5)


In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()  #4499 movies?

In [ ]:
mid = dataset[dataset['Rating'].isnull()]  #seperate movieIds
mid = mid.reset_index()
mid.head(5)

In [ ]:
mid.drop('Rating', axis=1, inplace=True)
mid['Cust_Id'] = mid['Cust_Id'].str[:-1]
mid.rename(columns={'Cust_Id':'Movie_Id'}, inplace=True)
mid.head(4)

In [ ]:
mid.tail(4)

In [ ]:
mid['Movie_Id'] = mid['Movie_Id'].astype(int)
mid.info()

In [ ]:
dataset.dropna(inplace=True)  #remove movieId rows
dataset.head(5)

In [ ]:
dataset['Movie_Id'] = None
start_idx=1
for i in range(1, mid.shape[0]):
    end_idx = mid.iloc[i]["index"] - 1
    dataset.loc[start_idx:end_idx, "Movie_Id"] = mid.iloc[i-1]["Movie_Id"]
    start_idx = end_idx + 1

dataset.loc[start_idx:, "Movie_Id"] = mid.iloc[mid.shape[0]-1]["Movie_Id"]
dataset.head(5)

In [ ]:
# WRONG RESULT, FOR LOOP CODE UPDATED, RERUN ABOVE ALL
dataset.tail(5)

In [ ]:
dataset[dataset["Movie_Id"]==4498]  #test if movie_ids applied good? ---seems good

In [ ]:
dataset.isnull().sum()

remove benchmark movies:

In [ ]:
benchmark_movies = dataset.groupby("Movie_Id")["Rating"].agg(["count"])

In [ ]:
benchmark = round(benchmark_movies["count"].quantile(0.6), 0)
benchmark

In [ ]:
in_movies = benchmark_movies[benchmark_movies["count"] >= benchmark].index

In [ ]:
in_movies

In [ ]:
dataset = dataset[dataset["Movie_Id"].isin(in_movies)]
dataset.shape

remove benchmark customers:

In [ ]:
benchmark_cust = dataset.groupby("Cust_Id")["Rating"].agg(["count"])

In [ ]:
benchmark = round(benchmark_cust["count"].quantile(0.6), 0)
benchmark

In [ ]:
in_cust = benchmark_cust[benchmark_cust["count"] >= benchmark].index

In [ ]:
dataset = dataset[dataset["Cust_Id"].isin(in_cust)]
dataset.shape

In [ ]:
#destroy huge object
#dataset = None
#del dataset

In [ ]:
df_genre = pd.read_csv("movie_titles.csv", encoding="latin-1", on_bad_lines='skip', header=None, names=["Id","year","name"])
df_genre.head(4)

In [ ]:
df_genre.nunique()

In [ ]:
df_genre.shape

In [ ]:
df_genre.info()

In [ ]:
df_genre.dropna(inplace=True)
#df_genre.isnull().sum()
df_genre.duplicated().sum()

In [ ]:
df_genre['year']=df_genre["year"].astype(int)
df_genre.head(4)

SVD model:

In [ ]:
!pip install scikit-surprise

In [ ]:
# restart the kernel after installation for the changes to take effect.

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
dataset[["Cust_Id","Movie_Id","Rating"]][:5]

In [ ]:
data = Dataset.load_from_df(dataset[["Cust_Id","Movie_Id","Rating"]][:1000000], Reader())
data

In [ ]:
model=SVD()

In [ ]:
cross_validate(model,data,measures=["RMSE"],cv = 3)

In [ ]:
dataset['Movie_Id'] = dataset['Movie_Id'].astype(int)
dataset['Cust_Id'] = dataset['Cust_Id'].astype(int)

In [ ]:
dataset.info()

RECOMMENDATION:

In [ ]:
#dataset.iloc[479281:479299]  #randome customer: 1867481
user = dataset[dataset["Cust_Id"]==1867481]

In [ ]:
user

In [ ]:
df_genre.info()

In [ ]:
df_genre = df_genre[df_genre["Id"].isin(in_movies)]
df_genre.shape

In [ ]:
df_genre["Estimated_Score"] = df_genre["Id"].apply(lambda x:model.predict(1867481, x).est)

In [ ]:
df_genre

In [ ]:
df_genre.sort_values("Estimated_Score", ascending=False).head(10)  #top 10 predictions for customer: 1867481